In [ ]:
import os

if os.getcwd().endswith('notebooks'):
    os.chdir('../../src/ml')

from load_matrix import load_df
from model_trainer import MLModel

In [2]:
df=load_df('gene_expression')
df.head()

,ILMN_2055271,ILMN_2383229,ILMN_1806310,ILMN_1653355,ILMN_2359168,ILMN_1735045,ILMN_1680754,ILMN_1755321,ILMN_1698554,ILMN_1814092,...,ILMN_1685547,ILMN_2348512,ILMN_1743643,ILMN_1656676,ILMN_2371169,ILMN_1701875,ILMN_1786396,ILMN_1653618,ILMN_2137536,disease_status
label,,,,,,,,,,,,,,,,,,,,,
Sample_GSM1317896,7.478478,7.732890,7.346000,7.541464,7.080655,7.190928,7.043338,7.282018,7.467964,7.253072,...,7.085921,7.725375,7.403544,9.052076,10.826351,11.510887,7.748866,7.265069,7.520050,0
Sample_GSM1317897,7.194329,7.091616,7.127377,7.139050,7.076957,7.101918,7.574589,7.151289,7.164247,7.159713,...,7.336513,7.182846,7.689322,10.352952,11.409938,12.015053,7.996690,7.173827,7.241760,0
Sample_GSM1317898,7.118431,7.008995,7.147905,7.251543,7.031998,7.064355,7.070183,7.306010,7.277900,7.067508,...,7.122465,7.274913,7.498920,9.439990,10.910384,11.717704,8.072896,7.261295,7.426808,0
Sample_GSM1317899,7.286120,7.122940,7.106208,7.241961,7.050389,7.096496,7.110168,7.278481,7.289622,7.224060,...,7.104158,7.215575,7.570657,9.394814,10.858485,12.152800,7.796620,7.399535,7.567951,0
Sample_GSM1317900,7.225378,7.035687,7.268438,7.289307,7.099056,7.138676,7.375244,7.279981,7.543007,7.231372,...,7.223078,7.218124,7.576488,9.480348,11.233749,12.124186,7.987269,7.235961,7.218149,0


In [12]:
#count disease_status values
df['disease_status'].value_counts()

disease_status
1    127
0     36
Name: count, dtype: int64

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

X=df.drop(columns=['disease_status']).values
y=df['disease_status'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

grid_search_params ={'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}

svm_gs=GridSearchCV(estimator=SVC(), param_grid=grid_search_params, cv=3,verbose=2)

In [4]:
svm_gs.fit(X_train, y_train)
svm_gs.best_params_

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   2.1s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   8.1s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   1.6s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   1.6s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   1.7s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=  10.0s
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time=  18.1s
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time=   6.4s
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time=   0.1s
[CV] END ......................C=0.1, gamma=auto, kernel=rbf; total time=   3.1s
[CV] END ......................C=0.1, gamma=auto, kernel=rbf; total time=   3.6s
[CV] END ......................C=0.1, gamma=auto

{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}

In [5]:
y_pred=svm_gs.predict(X_test)
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

1.0

with class weights

In [10]:
model=SVC(C=10, gamma='scale', kernel='linear', class_weight='balanced')
model.fit(X_train,y_train)

,C,10
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,'balanced'
,verbose,False


In [11]:
y_pred=model.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

with MLModel